# Декораторы

# Задача 1: Суммы прогрессий

Необходимо реализовать функционал для подсчета суммы первых n + 1 - членов арифметической и геометрической прогрессии с возможностями настройки шага и значения первого члена. n соответствует числу вызовов функции по подсчету суммы. 

Предполагаемые сценарии использования:

```python

sum_arithmetic = make_arithmetic_progression_sum(first_member=2, step=0.5)
sum_geometric = make_geometric_progression_sum(first_member=1, step=0.5)

print(sum_arithmetic())
print(sum_arithmetic())
print('')
print(sum_geometric())
print(sum_geometric())
```

*Вывод*:
```console
4.5
7.5

1.5
1.75
```

In [5]:
def make_arithmetic_progression_sum(first_member: float, step: float):
    general_sum = first_member
    last_elem = first_member

    def sum_arithmetic():
        nonlocal general_sum
        nonlocal last_elem
        last_elem += step
        general_sum += last_elem
        return general_sum
    
    return sum_arithmetic

def make_geometric_progression_sum(first_member: float, step: float):
    general_product = first_member
    last_elem = first_member

    def product_geo():
        nonlocal general_product
        nonlocal last_elem
        last_elem *= step
        general_product += last_elem
        return general_product
    
    return product_geo

sum_arithmetic = make_arithmetic_progression_sum(first_member=2, step=0.5)
sum_geometric = make_geometric_progression_sum(first_member=1, step=0.5)

print(sum_arithmetic())
print(sum_arithmetic())
print('')
print(sum_geometric())
print(sum_geometric())
    




4.5
7.5

1.5
1.75


# Задача 2: Среднее

Предположим, что мы занимаемся инвистициями и у нас есть некоторый портфель акций. Каждый день наш портфель приносит нам некоторый доход или убыток. Мы задались целью: каждый день фиксированного периода определять средний доход (или убыток), который мы получаем. С этой целью мы реализовали функцию get_avg(), принимающую на вход значение заработка на сегодняшний день. Наша функция вычисляет среднее в течении определнного фиксированного периода, скажем, 30 дней, после чего обнуляется и начинает вычислять среднее заново, но уже по новым значениям. 

Также у нас есть друзья инвесторы, которые оценили разработанный нами функционал и хотели бы заполучить свой экземпляр функции get_avg, для подсчета своего дохода в течении интересующего их промежутка времени.

Ваша задача: реализовать функционал, для получения произвольного числа независимых функций get_avg(). В момент создания функции сообщается длительность периода расчета среднего, по достижении которого среднее начинает расчитываться заново, а также наш начальный доход. При каждом вызове функции передается число - заработок в текущий день.

Предполагаемые сценарии использования:

```python

get_avg1 = make_averager(accumulation_period=2)
print(get_avg1(78))
print(get_avg1(-17))
print(get_avg1(52))
```

*Вывод*:
```console
78.0
30.5
52.0
```

In [13]:
def make_averager(accumulation_period: int):
    amount = 0
    general_sum = 0

    def get_avg1(elem: float):
        nonlocal amount
        nonlocal general_sum
        amount += 1
        general_sum += elem
        mean = general_sum / amount
        if amount == accumulation_period:
            amount = 0
            general_sum = 0
        
        return mean
    
    return get_avg1
    
get_avg1 = make_averager(accumulation_period=2)
print(get_avg1(78))
print(get_avg1(-17))
print(get_avg1(52))


78.0
30.5
52.0


# Задача 3: Сбор статистик

Предположим, что мы работаем в отделе аналитики некоторой компании. В компании также существуют другие отделы, которые разрабатывают некоторые функции для осуществления сложных вычислений. Также в нашей компании существует отдел планирования, который следит за исполнением сроков реализации той или иной функции, и в случае, если разработка затягивается, начинает торопить разработчиков. В таком случае разработчики пишут медленный код на скорую руку, что расстраивает заказчиков.

Наша задача, как аналитиков, собрать статистику по проблемным функциям. Нас интересует количество вызовов функции, а также среднее время выполнения функции. Все статистики собираются в отдельную базу данных - специальный единый словарь. Более того, статистика должна собираться не для всех функций, а только для функций, зарегестрированных в базе данных. Затем эта информация будет передана начальству, чтобы в скорейшее время заняться переписанием долгих и популярных функций.

Ваша задача реализовать функционал для регистрации функций в БД и сбора статистик. 

In [23]:
from time import time, sleep
from functools import wraps
from random import randint

functions_register = {}

def register(func):
    global functions_register
    if func not in functions_register:
        functions_register[func.__name__] = {"calls_amount": 0, "time_avg": 0}
    pass

def statistic_deko(func):
    register(func)

    @wraps(func)
    def func_with_stats(*args, **kwargs):
        global functions_register
        time_start = time()

        result = func(*args, **kwargs)

        time_finish = time()
        work_time = time_finish - time_start
        total_time = functions_register[func.__name__]["time_avg"] * functions_register[func.__name__]["calls_amount"]
        functions_register[func.__name__]["calls_amount"] += 1
        new_total_time = total_time + work_time

        functions_register[func.__name__]["time_avg"] = new_total_time / functions_register[func.__name__]["calls_amount"]

        return result
    
    return func_with_stats

@statistic_deko
def rand_sleep():
    t = randint(1, 4)
    sleep(t)
    return t

In [26]:
print(rand_sleep(), rand_sleep(), rand_sleep(), rand_sleep())
print(functions_register)

4 4 4 3
{'rand_sleep': {'calls_amount': 12, 'time_avg': 3.000802834828695}}


# Задача 4: наивный LRU-кэш

Представим, что вы являетесь сотрудником департамента оптимизаций некоторой компании, занимающейся разработкой ПО для научных вычислений. Раздел аналитиков предоставил вам исследование, согласно которому значительная часть функций в вашей компании работает очень медленно и должна быть оптимизирована. Более того, согласно исследованию, вызовы этих функций в основном осуществляются с ограниченным множеством аргументов. Для оптимизации этих функций вы решили использовать LRU-кэш:

- вы заранее фиксируете размер кэша - памяти, выделенной для хранения результатов вычислений функции;  
- в кэше хранится следующая информация: аргументы вызова - результат;  
- помимо этого для каждой пары хранится время их последнего вызова;  
- в случае достижения объема кэша установленной границы удаляются значения, чьи времена последних вызовов являются самыми старыми;  

Для применения данного подхода ко всем проблемным функциям, не переписывая сами функции, вы решили реализовать параметрический декоратор, т.к. для разных функций требуется разный размер кеша.

Ваша задача: реализовать описанный декоратор.

In [42]:
from time import time


def LRU(cache_size: int = 10):

    def start_caching():
        LRU_cache = {}

        def dec_cache(func):

            def caching(*args, **kwargs):
                nonlocal LRU_cache

                argument = args + tuple(kwargs.values()) 
                if argument in LRU_cache:
                    time_func = round(time(), 10)
                    LRU_cache[argument]["last_call"] = time_func
                    return LRU_cache[argument]["value"]
                
                print(len(LRU_cache.values()))
                print(LRU_cache)
                if len(LRU_cache) > cache_size:
                    min_time = 10000000000000000000000000000
                    print("aaa")
                    for key in LRU_cache:
                        min_time = min(min_time, LRU_cache[key]["last_call"])
                    for key in LRU_cache:
                        if LRU_cache[key]["last_call"] == min_time:
                            print(LRU_cache[key])
                            del LRU_cache[key]
                            break

                time_func = round(time(), 10)
                result = func(*args, **kwargs)
                LRU_cache[argument] = {"value": result, "last_call": time_func}

                return result
            
            return caching
        return dec_cache
    return start_caching

    
@LRU(5)
def geo_progression(length):
    if length <= 2:
        return 1
    return geo_progression(length - 1) + geo_progression(length - 2)


TypeError: LRU.<locals>.start_caching() takes 0 positional arguments but 1 was given

In [45]:
def LRU(stek_size: int = 10):

    def LRU_sub(func):

        def cache(*args, **kwargs):
            nonlocal LRU_cache
            nonlocal length

            argument = args + tuple(kwargs.values())
            if argument in LRU_cache:
                time_call = time()
                LRU_cache[argument]["time_call"] = time_call
                return LRU_cache[argument]["value"]
            
            if length == stek_size:
                min_len = 100000000000000000000000
                for i in LRU_cache:
                    if min_len < LRU_cache[i]["time_call"]:
                        min_len = LRU_cache[i]["time_call"]
                        for_del = i
                del LRU_cache[for_del]
                length -= 1
            
            time_call = time()
            result = func(*args, **kwargs)
            LRU_cache[argument] = {"value": result, "time_call": time_call}
            length += 1

            return result

        return cache
    return LRU_sub



@LRU()
def geo_progression(length):
    if length <= 2:
        return 1
    return geo_progression(length - 1) + geo_progression(length - 2)

In [46]:
geo = geo_progression()
print(geo(1000))

TypeError: geo_progression() missing 1 required positional argument: 'length'

In [41]:
geo = geo_progression()
print(geo(1000))

0
{}


TypeError: LRU.<locals>.start_caching.<locals>.dec_cache() takes 0 positional arguments but 1 was given

In [34]:
print(f'counter1 closure: {geo.__closure__}')

for i, cell in enumerate(geo.__closure__):
    print(f'geo cell{i} content: {geo.cell_contents};')

AttributeError: 'int' object has no attribute '__closure__'